# LENDING CLUB CASE

In [ ]:
# Importing essential libs and modules for analysis
import pandas as pd #To work with dataset
import numpy as np #Mathematical operation rich lib
import seaborn as sns #Graphical library 
import matplotlib.pyplot as plt #Used as backbone for seaborn
import tk #as suggested due to some technical glitch from matplotlib lib
import tkinter #as suggested due to some technical glitch from matplotlib lib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Reading dataset
df = pd.read_csv('loan.csv')

In [ ]:
# Checking data for content
df.head()

In [ ]:
#Reading insights
df.describe()

In [ ]:
#Dropping Unnecesary Columns 
columns_droplist = ['delinq_2yrs','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util','total_acc','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','last_credit_pull_d','application_type']
df.drop(labels = columns_droplist, axis =1, inplace=True)

In [ ]:
#Checking for null values
df.isnull().sum()

In [ ]:
#Deleting columns with null values having more than 85% of null values
for col in df.columns:
    if df[col].isnull().sum() >= (len(df.index)*85)/100 :
        del df[col]
    

In [ ]:
#Checking updated null values
df.isnull().sum()

In [ ]:
#Check dataset info for dtypes and col number for reference 
df.info()

In [ ]:
#Using dtale for better insights as it's hoghly potent with great functionalities
import dtale
d = dtale.show(df)
d

## DATA CLEANING

 We see many useful columns have descriptive texts like emp_length, int_rate so we would clean data to make it meaningful

In [ ]:
df.emp_length.fillna('0',inplace=True) # Populating null values with 0
df['emp_length']=df.emp_length.str.extract('(\d+)') # Extracting numerical data

In [ ]:
df['emp_length'].head() # Results are as expected

In [ ]:
# Removing % symbol from relevant columns
df['int_rate'] = df['int_rate'].str.rstrip('%')

In [ ]:
df['int_rate'].head() # Results are as expected

In [ ]:
# Type conversion to numeric for columns that are numeric in nature for further analysis

cols = ['loan_amnt','funded_amnt','int_rate','funded_amnt_inv','installment','annual_inc','dti','emp_length']
df[cols] = df[cols].apply(pd.to_numeric)

 Since we got our data almost ready, let's look what are possible reasons for loans and what chunk do repay loan

In [ ]:
print((df.loan_status.value_counts()*100)/len(df))
print((df.purpose.value_counts()*100)/len(df))

It tells a lot about loan status
We see around 83% of loans are fully paid and 14% are Charged Off which is a lot in practical scenario

Major chunk of loan taken is against debt followed by credit 

In [ ]:
# Segregating loan issuance date data for analysis purpose
df.issue_d = pd.to_datetime(df.issue_d, format='%b-%y')
df['year']=df['issue_d'].dt.year
df['month']=df['issue_d'].dt.month

In [ ]:
#Term column contains month so exctracting integer values for correlation
df['term']=df.term.str.extract('(\d+)')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# Now we see that our analysis take place only for charged off and fully paid one's
# as current is dynamic so w'll retain only those values

loan = df[df.loan_status != "Current"]
loan.loan_status.unique()

In [ ]:
loan

We'll try to visualise some basic data to check for outliers as well as range

In [ ]:
sns.boxplot(x=loan['loan_amnt'])

In [ ]:
sns.boxplot(x=loan['annual_inc'])

In [ ]:
sns.boxplot(x=loan['int_rate'])

In [ ]:
sns.boxplot(x=loan['funded_amnt_inv'])

# We observe that there are some outliers in our main columns which are basics of personal wealth and loan amount received
We'll treat data further for outliers(if needed) and null values for correlations 

In [ ]:
print(loan.loan_amnt.quantile([0.75,0.90,0.95,0.97,0.975, 0.98, 0.99, 1.0]))
print(loan.funded_amnt_inv.quantile([0.75,0.90,0.95,0.97,0.975, 0.98, 0.99, 1.0]))
print(loan.annual_inc.quantile([0.75,0.90,0.95,0.97,0.975, 0.98, 0.99, 1.0]))

We observe that annual income needs to be treated for outliers 

In [ ]:
new_annual_inc = loan['annual_inc'].quantile(0.95)
loan = loan[loan.annual_inc <= new_annual_inc]

In [ ]:
sns.boxplot(x=loan['annual_inc'])

Now annual income data seams to be more streamlined

# Viasulaizing Bi/Categorical Data for analysis

In [ ]:
sns.countplot(x = 'loan_status', data = loan)

In [ ]:
#analysisng for home_ownership
loan['home_ownership'].unique()

In [ ]:
sum(loan['home_ownership'] == 'NONE')

In [ ]:
#replacing 'NONE' with 'OTHERS'
loan['home_ownership'].replace(to_replace = ['NONE'],value='OTHER',inplace = True)

In [ ]:
sns.countplot(x = 'home_ownership', data = loan)

In [ ]:
sns.countplot(x = 'home_ownership',data=loan[loan['loan_status']=='Charged Off'])
# Checking for Charged Off customers    

In [ ]:
sns.countplot(x = 'verification_status',data=loan[loan['loan_status']=='Charged Off'])

In [ ]:
sns.countplot(x = 'term',data=loan[loan['loan_status']=='Charged Off'])

In [ ]:
#Checking if funded amount/annual_income & ROI have any observations regarding defaulters
loan['funded_amnt_inv_bins'] = pd.cut(loan['funded_amnt_inv'], bins=7,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k']) 
sns.countplot(x = 'funded_amnt_inv_bins',data=loan[loan['loan_status']=='Charged Off'])

In [ ]:
loan['annual_inc_bins'] = pd.cut(loan['annual_inc'], bins=6,precision =0,labels =['5k-30k','31k-55k','56k-80k','81k-105k','106k-130k','130k-200k'])
sns.countplot(x = 'annual_inc_bins',data=loan[loan['loan_status']=='Charged Off'])

In [ ]:
loan['int_rate_bins'] = pd.cut(loan['int_rate'], bins=4,precision =0,labels=['5%-10%','10%-15%','15%-20%','20%-25%'])
sns.countplot(x = 'int_rate_bins',data=loan[loan['loan_status']=='Charged Off'])

In [ ]:
fig, ax = plt.subplots(figsize = (25,8))
sns.countplot(x = 'purpose',data=loan[loan['loan_status']=='Charged Off'])
# Checking for Charged Off customers
# using matplot as sns fig size was small to give clear picture

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
sns.countplot(x='month', data=loan[loan['loan_status']=='Charged Off'],ax=axes[0])
sns.countplot(x='year', data=loan[loan['loan_status']=='Charged Off'])

In [ ]:
#Since it is observed that 2011 saw a spike so analysing monthwise for that year to see any discrepancies
sns.countplot(x='month', data=loan[(loan['loan_status']=='Charged Off') & (loan['year']==2011)])

# There are solid observations that can be made with these visualisations alone but for broader aspect we'll create bins and analyse data

In [ ]:
loan.corr().style.background_gradient(cmap="Blues")

At this point correlations doesn't yield very fruitful results since we've already analysed most of columns
however there's still some things that we can some up

# SUMMARISING OUR OBSERVATIONS

# Defaulters can be pre-identified if

House_ownership is categorised as 'RENT'

If apllied loan to clear other debts

If ROI for loan provided falls under 10-15%

Income range of applicants fall under 31K - 55K

Term of 36 months

Loan sanctioned ranges between 5K-10K

Loans are being sanctioned in month of December 

Status of loan is Not verified

Purpose stated is 'debt_consolidation'
